# Outline
 Benchmarking different approaches will happeb here. Profiling might still happen in the main simulations notebook.

In [1]:
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import ipdb
import ipywidgets as widgets
from IPython.display import display
from tqdm import tqdm
import matplotlib.path as path
import matplotlib.patches as patches
import matplotlib.animation as animation
from IPython.display import HTML
from line_profiler import LineProfiler
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics.pairwise import euclidean_distances
from numba import jit

np.random.seed(52500)

# 1D benchmarking

In [2]:
def distance_multiplier(df, r1, r2, thresh, power):
    """
    returns a distance multiplier based on two individuals, to be smacked 
    onto the transmission rate, that is always less than 1. 
    
    if the metric distance (defined within) is below thresh, multiplier of 1.
    Anything greater falls off as distance to the power law. Power is some law greater than 0.
    
    Inputs:
        df : (pandas DataFrame) object holding all values of infected people. Each
                            column of "infected day _" corresponds to a different day, 
                            with "_" being some integer or float. The "name" column
                            assigns a name to each object, independent of index. In
                            the infected columns, a 0 counts as infected, while a 1 is 
                            healthy.
        r1 : (float) position of first point.
        r2 : (float) position of second point.
        thresh : (float) distance less than which infection is transmitted at the trans_rate;
                            that is, less than which this function returns a value of 1. At
                            a distance greater than this, this function returns 1/distance^power.
        power : (float) Greater than 0. Power to which the multiplier falls off if the distance
                            is greater than some threshold.
    
    Outputs:
        multiplier : (float) suppresses the rate of transmission.
    """
    def metric_distance(r1, r2):
        """
        Returns distance between two points.
        
        Inputs:
            r1 : (float) position of first point.
            r2 : (float) position of second point.
        
        Outputs:
            dist : (float) distance between the two points.
            
        """
        if type(r1) != float:
            raise AssertionError("r1 must be a float.")
        if type(r2) != float and type(r2) != np.float64 and type(r2) != np.float32:
            raise AssertionError("r2 must be a float.")
        
        dist = abs(r1 - r2)
        return dist
    # first check input types
    if type(thresh) != int and type(thresh) != float:
        raise AssertionError("wrong type for thresh.")
    if type(df) != pd.core.frame.DataFrame:
        raise AssertionError("df must be a pandas DataFrame.")
    if type(r1) != float:
        raise AssertionError("r1 must be a float.")

    if type(r2) != float and type(r2) != np.float64 and type(r2) != np.float32:
        raise AssertionError("r2 must be a float.")
    if type(power) != float and type(power) != int:
        raise AssertionError("power must be a float or integer.")
    
        
    # first check input values
    if thresh <= 0:
        raise AssertionError("thresh must be positive.")
    if power <= 0:
        raise AssertionError("power must be positive.")
        
    dist = metric_distance(r1, r2)
    if dist < thresh:
        return 1.
    else:
        return 1/pow(dist, power)

In [3]:
N = 10000
name = np.arange(N)
distrib_pop = np.random.uniform
kwargs_for_pop = {'low':-10, 'high':10}
locs = distrib_pop(size=N, **kwargs_for_pop)
zero_infected = np.ones(N)
d = {'name': np.arange(N), 'infected day 0': zero_infected, 'locs' : locs} 

df1D_test = pd.DataFrame(data=d)

In [ ]:
%%timeit
power = 2
thresh = 2

r2 = df1D_test['locs'][1]
df1D_test['locs'].apply(lambda x:distance_multiplier(df1D_test, x, r2, thresh, power)) 

In [ ]:
%%timeit

# this is certainly around 25% faster
power = 3
thresh = 2
r2 = df1D_test['locs'][1]
df1D_test['subs'] = df1D_test['locs'].apply(lambda x:abs(x - r2))
df1D_test['metrics'] = df1D_test['subs'].apply(lambda x:1/pow(x, power) if x > thresh else 1)

# 2D benchmarking

## 2D distance metric

In [50]:
N = 1000000
name = np.arange(N)
distrib_pop = np.random.uniform
kwargs_for_pop = {'low':-10, 'high':10}
y = distrib_pop(size=N, **kwargs_for_pop)
x = distrib_pop(size=N, **kwargs_for_pop)
test = [[1,2],[2,3]]
zero_infected = np.ones(N)
d = {'name': np.arange(N), 'infected day 0': zero_infected, 'x' : x, 'y' : y} 

df2D_test = pd.DataFrame(data=d)

In [ ]:
%%timeit
euclidean_distances(df2D_test[['x', 'y']], df2D_test[['x', 'y']])

In [ ]:
a = euclidean_distances(df2D_test[['x', 'y']], df2D_test[['x', 'y']])
a[1]

In [ ]:
%%timeit
euclidean_distances([df2D_test['x'], df2D_test['y']], [df2D_test['x'], df2D_test['y']])

In [ ]:
a = euclidean_distances([df2D_test['x'], df2D_test['y']], [df2D_test['x'], df2D_test['y']])
a[1]

In [ ]:
%%timeit
euclidean_distances([df2D_test['x'].values, df2D_test['y'].values], [df2D_test['x'].values, df2D_test['y'].values])

In [ ]:
a = euclidean_distances([df2D_test['x'].values, df2D_test['y'].values], [df2D_test['x'].values, df2D_test['y'].values])
a[1]

In [14]:
%%timeit
distances = pdist([df2D_test['x'].values, df2D_test['y'].values], metric='euclidean')
dist_matrix = squareform(distances)

184 µs ± 32.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [15]:
distances = pdist([df2D_test['x'].values, df2D_test['y'].values], metric='euclidean')
a = squareform(distances)
a[1]

array([809.84252975,   0.        ])

In [18]:
%%timeit
distances = pdist(np.concatenate((test, df2D_test[["x", "y"]].values)), metric='euclidean')
dist = squareform(distances)

3.17 s ± 398 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
distances = pdist(np.concatenate((test, df2D_test[["x", "y"]].values)), metric='euclidean')
a = squareform(distances)
a[1]

## Testing subtraction stuff

In [19]:
%%timeit
df2D_test['x'] - 1

502 µs ± 92.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [20]:
%%timeit
df2D_test['x'].sub(1)

544 µs ± 70.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [21]:
%%timeit
df2D_test['x']**2

561 µs ± 71.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [22]:
%%timeit
np.square(df2D_test['x'])

362 µs ± 98.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [23]:
%%timeit
np.power(df2D_test['x'], 2)

537 µs ± 110 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [24]:
%%timeit
np.power(df2D_test['x'].values, 2)

695 µs ± 37.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## testing cKTTree
- don't calculate all the distances
- cut off within a certain region

In [ ]:
from scipy.spatial import cKDTree
tree = cKDTree(df2D_test[["x", "y"]])
pairs = tree.query_pairs(2, p=2)   # 50km radius, L2 (Euclidean) norm

## Try the smart indexing

In [6]:
def distance(frame, ind1, ind2):
    x1, y1 = frame['x'].values[ind1], frame['y'].values[ind1]
    x2, y2 = frame['x'].values[ind2], frame['y'].values[ind2]
    return sqrt((x1-x2)**2 + (y1-y2)**2)

In [7]:
from math import sqrt

In [51]:
# %%timeit
# make empty distance matrix 
# and then populate the distance matrix if below threshold
thresh = 2
test_vals = np.round(np.linspace(1, 1000, 100))
df2D_test['Rank'] = df2D_test.x.rank() + df2D_test.y.rank()
df_sorted = df2D_test.sort_values('Rank', ascending=False).drop('Rank',axis=1)
dists = np.array([distance(df_sorted, 100, int(val)) for val in test_vals])
farthest_calc = int(test_vals[np.argmin(np.abs(dists - thresh))])
farthest_calc

879

In [46]:
# thresh = 2
# test_vals = np.round(np.linspace(1, 1000, 100))
# df2D_test['Rank'] = df2D_test.x.rank() + df2D_test.y.rank()
# df_sorted = df2D_test.sort_values('Rank', ascending=False).drop('Rank',axis=1)
# dists = np.array([distance(df_sorted, 100, int(val)) for val in test_vals])
# farthest_calc = int(test_vals[np.argmin(np.abs(dists - thresh))])
# farthest_calc

In [152]:
@jit(nopython=True)
def new_where(lst, index):
    return np.argwhere(lst == 200)[0][0]

In [156]:
%%timeit

# %%cython
def do_multiplier(df_sorted, df2D_test):
    x = 1.
    y = 2.
    power=3
    thresh = 3
    place_in_sorted = new_where(np.array(df_sorted.index), 200)
    calc_indices = df_sorted.index[place_in_sorted - np.int(909):place_in_sorted + np.int(909)]
    df_calc_x = df2D_test.x[calc_indices]
    df_calc_y = df2D_test.y[calc_indices]
    dists = np.sqrt((df_calc_x.sub(x))**2 + (df_calc_y.sub(y))**2)
    multiplier_col = np.where(dists > thresh, 1/np.power(dists, power), np.ones_like(dists))
    return multiplier_col
do_multiplier(df_sorted, df2D_test)


22.6 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [139]:
from line_profiler import LineProfiler

In [142]:
# %%timeit

# %%cython
def do_multiplier(df_sorted, df2D_test):
    x = 1.
    y = 2.
    power=3
    thresh = 3
    place_in_sorted = np.argwhere(df_sorted.index == 200)[0][0] # the index in the sorted array
    calc_indices = df_sorted.index[place_in_sorted - np.int(909):place_in_sorted + np.int(909)]
    df_calc_x = df2D_test.x[calc_indices]
    df_calc_y = df2D_test.y[calc_indices]
    dists = np.sqrt((df_calc_x.sub(x))**2 + (df_calc_y.sub(y))**2)
    multiplier_col = np.where(dists > thresh, 1/np.power(dists, power), np.ones_like(dists))
    return multiplier_col
%prun do_multiplier(df_sorted, df2D_test)



In [157]:
lp = LineProfiler()
lp_wrapper = lp(do_multiplier)
lp_wrapper(df_sorted, df2D_test)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.015582 s
File: <ipython-input-142-ffe3e877aaa3>
Function: do_multiplier at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def do_multiplier(df_sorted, df2D_test):
     5         1          3.0      3.0      0.0      x = 1.
     6         1          2.0      2.0      0.0      y = 2.
     7         1          1.0      1.0      0.0      power=3
     8         1          0.0      0.0      0.0      thresh = 3
     9         1       6792.0   6792.0     43.6      place_in_sorted = np.argwhere(df_sorted.index == 200)[0][0] # the index in the sorted array
    10         1        518.0    518.0      3.3      calc_indices = df_sorted.index[place_in_sorted - np.int(909):place_in_sorted + np.int(909)]
    11         1       1945.0   1945.0     12.5      df_calc_x = df2D_test.x[calc_indices]
    12         1       1904.0   1904.0     12.2      df_calc_y = df2D_test.y[calc_indices]
    13      

In [218]:
import itertools as IT
import collections

def get_indices_simple(data):
    return [np.where(data == i) for i in range(0, data.max() + 1)]

In [123]:
@jit(nopython=True)
def where(df_calc_x, df_calc_y, thresh, power, x, y):
    dists = np.sqrt(np.square(df_calc_x - x) + np.square(df_calc_y - y))
    return np.where(dists > thresh, np.power(dists, -power), np.ones_like(dists))

# This function is the winner!

In [47]:
@jit(nopython=True)
def find_first(searched, vec):
    """return the index of the first occurence of item in vec"""
    for i, item in enumerate(vec):
        if searched == item:
            return i
    return -1

In [48]:
def do_multiplier(df_sorted, df2D_test):
    x = np.float64(1)
    y = np.float64(2)
    power = np.float64(2)
    thresh = np.float64(2)
    place_in_sorted = find_first(0.046916200165345145, df_sorted.x.values)
    calc_indices = df_sorted.index[place_in_sorted - np.int(900):place_in_sorted + np.int(900)]
    df_calc_x = df2D_test.x[calc_indices]
    df_calc_y = df2D_test.y[calc_indices]
    dists = np.sqrt((df_calc_x.sub(x))**2 + (df_calc_y.sub(y))**2)
    multiplier_col = np.where(dists > thresh, np.power(dists, -power), np.ones_like(dists))
    return multiplier_col

In [49]:
%%timeit
do_multiplier(df_sorted, df2D_test)

3.36 ms ± 336 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [42]:
4e-3*5e6/(60 * 60)

5.555555555555555

In [38]:
lp = LineProfiler()
lp_wrapper = lp(do_multiplier)
lp_wrapper(df_sorted, df2D_test)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.012079 s
File: <ipython-input-36-953f02cecf9e>
Function: do_multiplier at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def do_multiplier(df_sorted, df2D_test):
     2         1          6.0      6.0      0.0      x = np.float64(1)
     3         1          2.0      2.0      0.0      y = np.float64(2)
     4         1          2.0      2.0      0.0      power = np.float64(2)
     5         1          1.0      1.0      0.0      thresh = np.float64(2)
     6         1       1248.0   1248.0     10.3      place_in_sorted = find_first(0.046916200165345145, df_sorted.x.values)
     7         1        377.0    377.0      3.1      calc_indices = df_sorted.index[place_in_sorted - np.int(900):place_in_sorted + np.int(900)]
     8         1       2171.0   2171.0     18.0      df_calc_x = df2D_test.x[calc_indices]
     9         1       2678.0   2678.0     22.2      df_calc_y = df2D_test.y[

In [132]:
%%timeit

def do_multiplier(df_sorted, df2D_test):
    x = np.float64(1)
    y = np.float64(2)
    power = np.float64(3)
    thresh = np.float64(3)
    place_in_sorted = np.int(np.argwhere(df_sorted.index == 100)[0][0]) # the index in the sorted array
    calc_indices = df_sorted.index[place_in_sorted - np.int(800):place_in_sorted + np.int(800)]
    df_calc_x = df2D_test.x[calc_indices]
    df_calc_y = df2D_test.y[calc_indices]
    dists = np.sqrt(np.square(df_calc_x.sub(x)) + np.square(df_calc_y.sub(y))).values
    multiplier_col = np.piecewise(dists, [dists < thresh, dists >= thresh], 
                                  [lambda x: x, lambda x: 1/np.power(x, power)])
    return multiplier_col
do_multiplier(df_sorted, df2D_test)

13.2 ms ± 933 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [190]:
(6e-3*5e6) / (60 * 60)

8.333333333333334

In [40]:
do_multiplier(df_sorted, df2D_test)


array([0.00041412, 0.0004219 , 0.00041605, ..., 0.00042614, 0.00042598,
       0.00041272])

In [116]:
(3.06e-3 * 10e6) / (60 * 60)

8.499999999999998

In [100]:
place_in_sorted - int(farthest_calc)

90230

## testing distance speed


In [133]:
%%timeit
dists = np.sqrt(np.square(df_calc_x - x) + np.square(df_calc_y - y))

NameError: name 'df_calc_x' is not defined

In [120]:
%%timeit
dists = np.sqrt((df_calc_x - x)**2 + (df_calc_y - y)**2)

823 µs ± 112 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [121]:
%%timeit
dists = np.sqrt((df_calc_x.sub(x))**2 + (df_calc_y.sub(y))**2)

755 µs ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Testing multiplier

In [125]:
%%timeit
multiplier_col = np.where(dists > thresh, 1/np.power(dists, power), np.ones_like(dists))

572 µs ± 24.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [126]:
%%timeit
multiplier_col = np.where(np.greater(dists, thresh), 1/np.power(dists, power), np.ones_like(dists))

569 µs ± 2.56 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Faster where

In [222]:
%timeit np.argwhere(df_sorted.index == 100)[0][0]

4.65 ms ± 305 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [231]:
next(find(df_sorted.index, lambda arr: arr == 100))

((770146,), 100)

In [146]:
%timeit find_first(9.897408695319996, df_sorted.x.values)

7.29 µs ± 87 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [147]:
find_first(9.897408695319996, df_sorted.x.values)

200

In [148]:
df_sorted.x.values[200] == 9.897408695319996

True

In [145]:
find_first(0.9917446504600829, df_sorted.x.values)

3194604

In [141]:
df_sorted.x.values[3194604]

0.9917446504600829

In [48]:
np.searchsorted(df_sorted.x.values, 6., side='right')

5000000

In [59]:
df_sorted.x.searchsorted(-3)

0

In [88]:
%%timeit
e = df_sorted.x.values
e.size - np.searchsorted(e[::-1], -1.7136843497278083, side = "right")

12.5 µs ± 525 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [116]:
e = df_sorted.x
np.searchsorted(e[::-1], 0.9917446504600829, side = "left")

1635571

In [117]:
df_sorted.x[1635571]

-0.8787600990392477

In [81]:
df_sorted.x.values[311]

9.841383311759135

In [123]:
e.size - df_sorted.x.searchsorted(2)

1602801

In [124]:
df_sorted.x.values[1602801]

5.648708923693695

In [127]:
np.diff(df_sorted.x)>=0

array([ True, False, False, ...,  True, False, False])

In [133]:
np.searchsorted

<function numpy.searchsorted(a, v, side='left', sorter=None)>

## Testing out square root

In [53]:
%timeit np.sqrt(df_sorted.x.sub(1)**2 + df_sorted.y.sub(1)**2)

24.6 ms ± 176 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [52]:
%timeit (df_sorted.x.sub(1)**2 + df_sorted.y.sub(1)**2).apply(np.sqrt)

24.6 ms ± 244 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [58]:
def f(x):
    x1, y1 = x
    return sqrt((x1 - 1)**2 + (y1 - 1)**2)

In [59]:
%timeit df_sorted[['x', 'y']].apply(lambda x: f(x), axis=1)

10.1 s ± 1.3 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
